In [14]:
import itertools
import mdtraj as md
import mdtraj.testing



In [67]:

import os
input_path = './data/pdbs'
output = './data/pqrs'


def pdb2pqr(input_path,pdb_files, output_path):
    import os
    if pdb_files is None:
        pdb_files = [f for f in os.listdir(input_path) if f.endswith('.pdb')]
    if isinstance(pdb_files, str):
        pdb_files = [pdb_files]
    for pdb_file in pdb_files:
        name = f'{(pdb_file.split('.')[0]).split('-')[1]}.pqr'
        os.system(f'pdb2pqr "{os.path.join(input_path, str(pdb_file))}" "{os.path.join(output_path, name)}" -ff={'AMBER'} --noop')


pdb2pqr(input_path, 'AF-P39846-F1.pdb', output)